In [5]:
import numpy as np


In [18]:
a=[np.array([0,0,0])]
b=np.array([0,0,2])

In [ ]:
np.(b)

TypeError: remainder() takes from 2 to 3 positional arguments but 1 were given

In [21]:
a=a+a

In [22]:
a

[array([0, 0, 0]), array([0, 0, 0])]

In [24]:
0.1/np.pi*180

5.729577951308232